## Further improvements

In [1]:
import arcpy

In [2]:
targetFC = r"C:\Demos\PYTS\SanDiego.gdb\CrimeTest"
inputTable = r"C:\Demos\PYTS\SanDiego.gdb\Crime_XY"

In [3]:
# Get a list of input fields
fieldMapDict = {}
fieldNamesTarget = []
for field in arcpy.Describe(targetFC).fields:
    if field.type != "OID" and field.type != "Geometry": # Don't include the OID or Shape fields
        fieldNamesTarget.append(field)

fieldMapDict[targetFC]=fieldNamesTarget

fieldNamesInput = []
for field in arcpy.Describe(inputTable).fields:
    if "OID" not in field.name and field.type != "OID": # Don't include the OBJECTID or OID_ fields
        fieldNamesInput.append(field)

fieldMapDict[inputTable]=fieldNamesInput


In [4]:
matchFields = ""
fieldMaps = []

for i in range(len(fieldNamesTarget)): # length of the list of field names for target fc (or input)
    # Get the paired fields
    targetField = fieldMapDict[targetFC][i] # Get a field from the target FC
    inField = fieldMapDict[inputTable][i] # Get a field from the input table
    print(inField.name, targetField.name) # Print the fields being added to a field map
    # Create the empty FieldMap object
    fm = arcpy.FieldMap() # create empty field map
    # Add the input and target dataset's paired fields
    fm.addInputField(targetFC, targetField.name) # Add the target field to field map
    fm.addInputField(inputTable, inField.name) # Add the input table field to field map
    # Add the FieldMap to the list of field maps
    fieldMaps.append(fm) # append field map to the list
    # Format a string to satisfy the append tool's match_field parameter
    match = f"{fm.getInputFieldName(0)} {fm.getInputFieldName(1)};" 
    matchFields += match

matchFields = matchFields[:-1] # Remove the last character ";"....it's not needed
# Create the FieldMappings object
fms = arcpy.FieldMappings()
# Add each FieldMap to the FieldMappings object
for fieldMap in fieldMaps:
    fms.addFieldMap(fieldMap)

print(f"There are {len(fieldMaps)} FieldMaps created:")
print(matchFields)

start_date EventTimeDate
X_WGS1984 XCoordinate
Y_WGS1984 YCoordinate
There are 3 FieldMaps created:
EventTimeDate start_date;XCoordinate X_WGS1984;YCoordinate Y_WGS1984


In [5]:
arcpy.management.Append(
    inputs=inputTable,
    target=targetFC,
    schema_type="NO_TEST",
    field_mapping=fms,
    subtype="",
    expression="",
    match_fields=matchFields,
    update_geometry="NOT_UPDATE_GEOMETRY"
)

<Result 'C:\\Demos\\PYTS\\SanDiego.gdb\\CrimeTest'>